# Notebook 03: Difference-in-Differences Estimation

**Goal:** Estimate causal effect of economic conditions on subscription rates using DiD methodology

---

## Setup


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import sys

sys.path.append('../src')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

figures_path="../outputs/figures/"

## 1. Load Clean Analysis Sample


In [ ]:
# Load final sample from notebook 02
df = pd.read_csv('../data/processed/analysis_sample.csv')
df['contact_date'] = pd.to_datetime(df['contact_date'])

print(f"{'='*70}")
print(f"ANALYSIS SAMPLE SUMMARY")
print(f"{'='*70}")
print(f"Total customers: {len(df):,}")
print(f"Wave 1 (May-Aug 2008, Pre-crisis): {(df['wave']=='wave_1').sum():,}")
print(f"Wave 2 (Apr-Aug 2009, Crisis recovery): {(df['wave']=='wave_2').sum():,}")

print(f"\nOutcome distribution:")
print(df.groupby('wave')['y_binary'].describe())

print(f"\nSample characteristics:")
print(f"✓ Fresh prospects only (previous=0, pdays=999)")
print(f"✓ Single-wave customers (no cross-contamination)")
print(f"✓ First contact per customer (customer-level analysis)")

FileNotFoundError: [Errno 2] No such file or directory: '..data/processed/analysis_sample.csv'

## 2. Simple Difference Estimation (Baseline)


### 2.1 Calculate Naive Difference


In [ ]:
print(f"\n{'='*70}")
print(f"NAIVE DIFFERENCE (NO CONTROLS)")
print(f"{'='*70}")

# Calculate mean subscription rates by wave
wave_1_rate = df[df['wave']=='wave_1']['y_binary'].mean()
wave_2_rate = df[df['wave']=='wave_2']['y_binary'].mean()
naive_diff = wave_2_rate - wave_1_rate

# Sample sizes
n_wave1 = (df['wave']=='wave_1').sum()
n_wave2 = (df['wave']=='wave_2').sum()

# Standard error of difference
se_wave1 = df[df['wave']=='wave_1']['y_binary'].sem()
se_wave2 = df[df['wave']=='wave_2']['y_binary'].sem()
se_diff = np.sqrt(se_wave1**2 + se_wave2**2)

# 95% Confidence interval
ci_lower = naive_diff - 1.96 * se_diff
ci_upper = naive_diff + 1.96 * se_diff

# T-test
t_stat, p_val = stats.ttest_ind(
    df[df['wave']=='wave_2']['y_binary'],
    df[df['wave']=='wave_1']['y_binary']
)

print(f"\nWave 1 (Pre-Crisis):")
print(f"  Subscription rate: {wave_1_rate:.4f} ({wave_1_rate*100:.2f}%)")
print(f"  Sample size: {n_wave1:,}")

print(f"\nWave 2 (Crisis Recovery):")
print(f"  Subscription rate: {wave_2_rate:.4f} ({wave_2_rate*100:.2f}%)")
print(f"  Sample size: {n_wave2:,}")

print(f"\nNaive Difference (Wave 2 - Wave 1):")
print(f"  Point estimate: {naive_diff:.4f} ({naive_diff*100:.2f} percentage points)")
print(f"  Standard error: {se_diff:.4f}")
print(f"  95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"  Relative increase: {(naive_diff/wave_1_rate)*100:.1f}%")

print(f"\nStatistical Test:")
print(f"  T-statistic: {t_stat:.3f}")
print(f"  P-value: {p_val:.6f}")
print(f"  Significant at 5%: {'Yes ✅' if p_val < 0.05 else 'No ❌'}")

# Store for later comparison
results_dict = {
    'Simple Difference': {
        'estimate': naive_diff,
        'se': se_diff,
        'p_value': p_val,
        'n': len(df)
    }
}

### 2.2 Visualize Simple Difference


In [ ]:
# Create comparison bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=['Wave 1<br>(Pre-Crisis)<br>May-Aug 2008', 
       'Wave 2<br>(Crisis Recovery)<br>Apr-Aug 2009'],
    y=[wave_1_rate * 100, wave_2_rate * 100],
    text=[f"{wave_1_rate*100:.1f}%", f"{wave_2_rate*100:.1f}%"],
    textposition='outside',
    marker=dict(
        color=['lightblue', 'salmon'],
        line=dict(color='darkgray', width=2)
    ),
    error_y=dict(
        type='data',
        array=[se_wave1*100*1.96, se_wave2*100*1.96],
        visible=True,
        color='black'
    )
))

# Add difference annotation
fig.add_annotation(
    x=0.5, y=max(wave_1_rate, wave_2_rate) * 50,
    text=f"<b>Δ = +{naive_diff*100:.1f}pp</b><br>(+{(naive_diff/wave_1_rate)*100:.0f}%)",
    showarrow=False,
    font=dict(size=16, color="darkgreen"),
    bgcolor="lightyellow",
    bordercolor="darkgreen",
    borderwidth=2,
    opacity=0.9
)

fig.update_layout(
    title='Subscription Rates: Fresh Prospects Only (Naive Comparison)',
    yaxis_title='Subscription Rate (%)',
    height=500,
    showlegend=False,
    yaxis=dict(range=[0, max(wave_1_rate, wave_2_rate)*120])
)

fig.show()
fig.write_image(f"{figures_path}09_subscription_rates_naive_comparison.png")

print("\nInterpretation:")
print("This is the NAIVE difference - it doesn't control for any confounders.")

## 3. Regression-Based DiD Estimation


### 3.1 Model 1: Simple Regression (No Controls)


**Formula:** `y_binary ~ wave_2`

Where `wave_2 = 1` if Wave 2, `0` if Wave 1


In [ ]:
print(f"\n{'='*70}")
print(f"MODEL 1: SIMPLE DiD REGRESSION (NO CONTROLS)")
print(f"{'='*70}")

# Create treatment indicator
df['wave_2'] = (df['wave'] == 'wave_2').astype(int)

# Model: y_binary ~ wave_2
# Coefficient on wave_2 = DiD estimate (same as simple difference)
model1 = smf.ols('y_binary ~ wave_2', data=df).fit()

print("\nRegression Results:")
print(model1.summary().tables[1])

# Extract key statistics
coef1 = model1.params['wave_2']
se1 = model1.bse['wave_2']
pval1 = model1.pvalues['wave_2']
r2_1 = model1.rsquared
n1 = int(model1.nobs)

print(f"\n{'='*70}")
print(f"Key Statistics:")
print(f"{'='*70}")
print(f"Coefficient (wave_2): {coef1:.4f}")
print(f"Standard error: {se1:.4f}")
print(f"95% CI: [{coef1 - 1.96*se1:.4f}, {coef1 + 1.96*se1:.4f}]")
print(f"P-value: {pval1:.6f}")
print(f"R-squared: {r2_1:.4f}")
print(f"N: {n1:,}")

print(f"\nInterpretation:")
print(f"The coefficient {coef1:.4f} is identical to the naive difference.")
print(f"This confirms: simple regression = difference in means when no controls.")

# Store results
results_dict['Model 1 (No controls)'] = {
    'estimate': coef1,
    'se': se1,
    'p_value': pval1,
    'n': n1,
    'r2': r2_1
}

### 3.2 Model 2: Control for Campaign Intensity


**Formula:** `y_binary ~ wave_2 + campaign`
Campaign intensity differed between waves (2.94 vs 2.32). Control for this potential confounder.


In [ ]:
print(f"\n{'='*70}")
print(f"MODEL 2: CONTROL FOR CAMPAIGN INTENSITY")
print(f"{'='*70}")

# Recall: Campaign intensity differed between waves (2.94 vs 2.32)
# If more contacts → higher conversion, this confounds our estimate

# Model: y_binary ~ wave_2 + campaign
model2 = smf.ols('y_binary ~ wave_2 + campaign', data=df).fit()

print("\nRegression Results:")
print(model2.summary().tables[1])

coef2 = model2.params['wave_2']
se2 = model2.bse['wave_2']
pval2 = model2.pvalues['wave_2']
r2_2 = model2.rsquared

campaign_coef = model2.params['campaign']
campaign_pval = model2.pvalues['campaign']

print(f"\n{'='*70}")
print(f"Key Statistics:")
print(f"{'='*70}")
print(f"Wave 2 coefficient: {coef2:.4f} (SE: {se2:.4f})")
print(f"Campaign coefficient: {campaign_coef:.4f} (p={campaign_pval:.4f})")
print(f"R-squared: {r2_2:.4f}")

print(f"\nChange from Model 1:")
print(f"  Model 1 (no controls): {coef1:.4f}")
print(f"  Model 2 (+ campaign): {coef2:.4f}")
print(f"  Difference: {coef2 - coef1:.4f} ({((coef2-coef1)/coef1)*100:.1f}%)")

if abs(coef2 - coef1) < 0.01:
    print(f"  ✅ Estimate barely changed - campaign is NOT a major confounder")
else:
    print(f"  ⚠️  Estimate changed - campaign intensity matters")

print(f"\nCampaign Effect:")
if campaign_pval < 0.05:
    print(f"  Each additional contact {'increases' if campaign_coef > 0 else 'decreases'} ")
    print(f"  subscription probability by {abs(campaign_coef):.4f} ({abs(campaign_coef)*100:.2f}pp)")
else:
    print(f"  Campaign intensity not statistically significant")

# Store results
results_dict['Model 2 (+ campaign)'] = {
    'estimate': coef2,
    'se': se2,
    'p_value': pval2,
    'n': int(model2.nobs),
    'r2': r2_2
}

### 3.3 Model 3: Add Customer Demographics


**Formula:** `y_binary ~ wave_2 + campaign + age + C(job) + C(marital) + C(education)`

Add customer characteristics to check if selection bias affects results.

In [ ]:
print(f"\n{'='*70}")
print(f"MODEL 3: ADD CUSTOMER DEMOGRAPHICS")
print(f"{'='*70}")

# Add customer characteristics to check for selection bias
# If Wave 1 and Wave 2 customers differ systematically, this matters

# Model: y_binary ~ wave_2 + campaign + age + job + marital + education
model3 = smf.ols('''y_binary ~ wave_2 + campaign + age + 
                     C(job) + C(marital) + C(education)''', 
                 data=df).fit()

print("\nRegression Results (showing key coefficients):")
print(model3.summary().tables[1][:10])  # First 10 rows to avoid clutter

coef3 = model3.params['wave_2']
se3 = model3.bse['wave_2']
pval3 = model3.pvalues['wave_2']
r2_3 = model3.rsquared

print(f"\n{'='*70}")
print(f"Key Statistics:")
print(f"{'='*70}")
print(f"Wave 2 coefficient: {coef3:.4f} (SE: {se3:.4f})")
print(f"P-value: {pval3:.6f}")
print(f"R-squared: {r2_3:.4f}")

print(f"\nProgression of Estimates:")
print(f"  Model 1 (no controls):        {coef1:.4f}")
print(f"  Model 2 (+ campaign):         {coef2:.4f}")
print(f"  Model 3 (+ demographics):     {coef3:.4f}")
print(f"  Change (Model 1 → Model 3):   {coef3 - coef1:.4f} ({((coef3-coef1)/coef1)*100:.1f}%)")

if abs(coef3 - coef1) < 0.01:
    print(f"\n✅ EXCELLENT: Estimate is stable across specifications")
    print(f"   This suggests minimal selection bias - Wave 1 and Wave 2")
    print(f"   customers are comparable on observables")
else:
    print(f"\n⚠️  CAUTION: Estimate changed with controls")
    print(f"   This suggests some selection bias - customer characteristics")
    print(f"   differ between waves and affect outcomes")

# Store results
results_dict['Model 3 (+ demographics)'] = {
    'estimate': coef3,
    'se': se3,
    'p_value': pval3,
    'n': int(model3.nobs),
    'r2': r2_3
}

### 3.4 Model 4: Full Specification (Economic Indicators)


**Purpose:** Check if economic variables mediate the effect

**Formula:** `y_binary ~ wave_2 + campaign + age + C(job) + C(marital) + C(education) + emp.var.rate + euribor3m`

In [ ]:
print(f"\n{'='*70}")
print(f"MODEL 4: FULL SPECIFICATION (ECONOMIC INDICATORS)")
print(f"{'='*70}")

# CRITICAL: Economic indicators are highly correlated with wave_2
# This tests whether wave_2 effect is MEDIATED by economic conditions

# Clean column names (remove dots for formula)
df['emp_var_rate'] = df['emp.var.rate']
df['cons_price_idx'] = df['cons.price.idx']
df['cons_conf_idx'] = df['cons.conf.idx']

# Model: Full controls including economic indicators
model4 = smf.ols('''y_binary ~ wave_2 + campaign + age + 
                     C(job) + C(marital) + C(education) +
                     emp_var_rate + euribor3m''', 
                 data=df).fit()

print("\nRegression Results (key coefficients):")
print(model4.summary().tables[1][:15])

coef4 = model4.params['wave_2']
se4 = model4.bse['wave_2']
pval4 = model4.pvalues['wave_2']
r2_4 = model4.rsquared

econ_coef_emp = model4.params['emp_var_rate']
econ_coef_euribor = model4.params['euribor3m']

print(f"\n{'='*70}")
print(f"Key Statistics:")
print(f"{'='*70}")
print(f"Wave 2 coefficient: {coef4:.4f} (SE: {se4:.4f})")
print(f"P-value: {pval4:.6f}")
print(f"R-squared: {r2_4:.4f}")

print(f"\nEconomic Indicator Effects:")
print(f"  Employment var rate: {econ_coef_emp:.4f} (p={model4.pvalues['emp_var_rate']:.4f})")
print(f"  Euribor 3m rate:     {econ_coef_euribor:.4f} (p={model4.pvalues['euribor3m']:.4f})")

print(f"\nProgression of Estimates:")
print(f"  Model 3 (demographics only):      {coef3:.4f}")
print(f"  Model 4 (+ economic indicators):  {coef4:.4f}")
print(f"  Reduction: {coef3 - coef4:.4f} ({((coef3-coef4)/coef3)*100:.1f}%)")

print(f"\n{'='*70}")
print(f"INTERPRETATION:")
print(f"{'='*70}")
if coef4 < coef3 * 0.7:
    print(f"✓ Economic indicators STRONGLY mediate the wave_2 effect")
    print(f"  The wave_2 coefficient shrank by {((coef3-coef4)/coef3)*100:.0f}%")
    print(f"  This suggests economic conditions EXPLAIN the outcome difference")
else:
    print(f"✓ Wave_2 effect persists even after controlling for economics")
    print(f"  This suggests additional mechanisms beyond measured indicators")

# Store results
results_dict['Model 4 (+ economics)'] = {
    'estimate': coef4,
    'se': se4,
    'p_value': pval4,
    'n': int(model4.nobs),
    'r2': r2_4
}

### 3.5 Regression Table Comparison


In [ ]:
print(f"\n{'='*70}")
print(f"REGRESSION RESULTS SUMMARY")
print(f"{'='*70}")

# Create comparison table
results_table = pd.DataFrame({
    'Model 1\n(No controls)': [
        f"{results_dict['Model 1 (No controls)']['estimate']:.4f}",
        f"({results_dict['Model 1 (No controls)']['se']:.4f})",
        f"{results_dict['Model 1 (No controls)']['r2']:.4f}",
        f"{results_dict['Model 1 (No controls)']['n']:,}"
    ],
    'Model 2\n(+ Campaign)': [
        f"{results_dict['Model 2 (+ campaign)']['estimate']:.4f}",
        f"({results_dict['Model 2 (+ campaign)']['se']:.4f})",
        f"{results_dict['Model 2 (+ campaign)']['r2']:.4f}",
        f"{results_dict['Model 2 (+ campaign)']['n']:,}"
    ],
    'Model 3\n(+ Demographics)': [
        f"{results_dict['Model 3 (+ demographics)']['estimate']:.4f}",
        f"({results_dict['Model 3 (+ demographics)']['se']:.4f})",
        f"{results_dict['Model 3 (+ demographics)']['r2']:.4f}",
        f"{results_dict['Model 3 (+ demographics)']['n']:,}"
    ],
    'Model 4\n(+ Economics)': [
        f"{results_dict['Model 4 (+ economics)']['estimate']:.4f}",
        f"({results_dict['Model 4 (+ economics)']['se']:.4f})",
        f"{results_dict['Model 4 (+ economics)']['r2']:.4f}",
        f"{results_dict['Model 4 (+ economics)']['n']:,}"
    ]
}, index=['Wave 2 Coefficient', 'Standard Error', 'R-squared', 'N Observations'])

print("\n" + results_table.to_string())

# Visualize coefficient stability
estimates = [r['estimate'] for r in results_dict.values()]
ses = [r['se'] for r in results_dict.values()]
labels = list(results_dict.keys())

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=estimates,
    y=labels,
    mode='markers',
    marker=dict(size=12, color='darkblue'),
    error_x=dict(
        type='data',
        array=[1.96*se for se in ses],
        visible=True,
        color='black'
    ),
    name='Point Estimate'
))

fig.add_vline(x=0, line_dash="dash", line_color="gray")

fig.update_layout(
    title='DiD Estimate Across Specifications (with 95% CI)',
    xaxis_title='Estimated Effect (percentage points)',
    yaxis_title='',
    height=400,
    showlegend=False
)

fig.show()
fig.write_image(f"{figures_path}10_did_estimate_stability.png")